# Inicialização

In [1]:
import psycopg2
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm
import seaborn as sns
import numpy as np
import scipy

from imblearn.under_sampling import RandomUnderSampler
import imblearn.pipeline

#import warnings
#warnings.filterwarnings("ignore")

In [2]:
dict_connect = {
    "dbname" : "lista4",
    "user" : "postgres",
    "password": "123321",
    "host": "localhost"
}
conn = psycopg2.connect(**dict_connect)
raw_data = pd.read_sql('SELECT * FROM tb_whatsapp_messages', con=conn)
raw_data.info()

C:\Users\Pichau\AppData\Local\Temp\ipykernel_7832\3275608030.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_data = pd.read_sql('SELECT * FROM tb_whatsapp_messages', con=conn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407957 entries, 0 to 407956
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      407957 non-null  int64         
 1   date_message            407957 non-null  datetime64[ns]
 2   id_member_anonymous     407957 non-null  object        
 3   ddi                     407957 non-null  object        
 4   ddd                     406311 non-null  object        
 5   country                 407866 non-null  object        
 6   country_iso3            407866 non-null  object        
 7   state                   406311 non-null  object        
 8   id_group_anonymous      407957 non-null  object        
 9   media                   185568 non-null  object        
 10  media_type              187528 non-null  object        
 11  media_url               132949 non-null  object        
 12  has_media               407957

In [3]:
textos = raw_data["text_content_anonymous"]
textos.dropna(inplace = True)
textos = textos[(textos != 'None') & (textos != "")]

In [4]:
textos = textos[ textos.apply(lambda x: len(x.split()) > 15)]

In [5]:
#textos = textos.sample(100000, random_state = 23)

# Preprocessamento

In [6]:
import nltk
from nltk.corpus import stopwords
import re
from urllib.parse import urlparse
import spacy


nlp = spacy.load('pt_core_news_sm')
STOP_WORDS = set(stopwords.words('portuguese'))
RENOMEAR = {
    "combr" : "",
    "com" : "",
    "https": "",
    "www": ""
    
}

In [7]:
def get_named_entities(text):
    doc = nlp(text)
    
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def extract_root_domain(text):
    url_pattern = r'(https?://[^\s]+)'
    
    def get_root_domain(url):
        try:
            parsed_url = urlparse(url)
            domain_parts = parsed_url.netloc.split('.')
            
            if len(domain_parts) >= 2:
                root_domain = '.'.join(domain_parts[-2:])
            else:
                root_domain = parsed_url.netloc
            
            return root_domain.capitalize()
        except ValueError:
            return ''
    
    return re.sub(url_pattern, lambda match: get_root_domain(match.group(0)), text)
    
def preprocessamento_regras_assoc(texto):

    texto_filtrado = texto.replace("*", "").replace("\n", " ")
    
    texto_filtrado = extract_root_domain(texto_filtrado)

    texto_filtrado = re.sub(r'[^\w\s]', '', texto_filtrado)

    texto_filtrado = ' '.join([ent.text for ent in nlp(texto_filtrado).ents]).lower()

    for k, v in RENOMEAR.items():
        texto_filtrado = texto_filtrado.replace(k, v)

    return texto_filtrado


textos_processados = textos.apply(preprocessamento_regras_assoc)
textos_processados.dropna(inplace = True)
textos_processados = textos_processados[textos_processados != ""]

In [8]:
textos_processados = textos_processados[textos_processados.apply(lambda x: len(x.split()) > 3)]

In [9]:
textos_processados.to_csv("textos_processados.csv")
textos_processados.shape

(61586,)

In [10]:
#textos_processados = pd.read_csv("textos_processados.csv")

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 4000, binary = True, stop_words = list(STOP_WORDS))
bag_of_words = vectorizer.fit_transform(textos_processados)
df = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns=vectorizer.get_feature_names_out())

In [12]:
df.sum().sort_values(ascending = False).iloc[:30]

brasil                 10245
whatsapp                8727
lula                    5863
canal                   4624
bolsonaro               4131
israel                  3990
deus                    3519
hamas                   3499
instagram               3440
youtube                 3241
stf                     3187
pensandodireita         2581
pt                      2457
partilhe                2441
terrabrasilnoticias     2351
veja                    2343
paulo                   2235
portal                  2221
vídeo                   2185
rio                     2176
federal                 1994
assista                 1968
vamos                   1758
polícia                 1731
governo                 1713
jesus                   1670
facebook                1658
news                    1634
gaza                    1628
saiba                   1575
dtype: Sparse[int64, 0]

# Apriori

In [13]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)

rules = pd.DataFrame(association_rules(frequent_itemsets, metric="lift", min_threshold=2))
rules

C:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(180),(canal),0.011366,0.075082,0.011188,0.984286,13.109477,0.010334,58.858418,0.934339
1,(canal),(180),0.075082,0.011366,0.011188,0.149005,13.109477,0.010334,1.161739,0.998704
2,(consulte),(180),0.010619,0.011366,0.010473,0.986239,86.769266,0.010352,71.840721,0.999085
3,(180),(consulte),0.011366,0.010619,0.010473,0.921429,86.769266,0.010352,12.592118,0.999840
4,(alexandre),(moraes),0.018527,0.015409,0.010246,0.553024,35.888847,0.009960,2.202780,0.990487
...,...,...,...,...,...,...,...,...,...,...
337,(saiba),"(brazilnewsinforma, brazil, news, informa)",0.025574,0.012909,0.010197,0.398730,30.888296,0.009867,1.641678,0.993021
338,(brazilnewsinforma),"(saiba, informa, news, brazil)",0.013169,0.010246,0.010197,0.774353,75.577310,0.010062,4.386288,0.999936
339,(brazil),"(brazilnewsinforma, saiba, news, informa)",0.017553,0.010197,0.010197,0.580944,56.971323,0.010018,2.361980,1.000000
340,(news),"(brazilnewsinforma, saiba, informa, brazil)",0.026532,0.010197,0.010197,0.384333,37.690330,0.009927,1.607692,1.000000


# FP-Growth

In [14]:
from mlxtend.frequent_patterns import fpgrowth
frequent_itemsets = fpgrowth(df, min_support=0.01, use_colnames=True)

rules = pd.DataFrame(association_rules(frequent_itemsets, metric="lift", min_threshold=1))
rules

C:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(lula),(whatsapp),0.095200,0.141704,0.015685,0.164762,1.162718,0.002195,1.027606,0.154671
1,(whatsapp),(lula),0.141704,0.095200,0.015685,0.110691,1.162718,0.002195,1.017419,0.163051
2,(lula),(brasil),0.095200,0.166353,0.029049,0.305134,1.834258,0.013212,1.199724,0.502675
3,(brasil),(lula),0.166353,0.095200,0.029049,0.174622,1.834258,0.013212,1.096224,0.545579
4,(hamas),(israel),0.056815,0.064787,0.030656,0.539583,8.328507,0.026975,2.031228,0.932935
...,...,...,...,...,...,...,...,...,...,...
361,"(hamas, gaza)",(israel),0.014630,0.064787,0.010295,0.703663,10.861094,0.009347,3.155905,0.921408
362,"(israel, gaza)",(hamas),0.016546,0.056815,0.010295,0.622179,10.950984,0.009354,2.496378,0.923972
363,(hamas),"(israel, gaza)",0.056815,0.016546,0.010295,0.181195,10.950984,0.009354,1.201084,0.963421
364,(israel),"(hamas, gaza)",0.064787,0.014630,0.010295,0.158897,10.861094,0.009347,1.171522,0.970826


# Eclat

In [15]:
df_copy = df.copy()
df_copy = df_copy.apply(lambda col: pd.Series(col.to_numpy()).replace(0, np.nan))
df_copy = df_copy.apply(lambda col: pd.Series(col.to_numpy()).replace(1, col.name))

In [16]:
df_copy.columns = range(df_copy.shape[1])

In [18]:
from pyECLAT import ECLAT

eclat_instance = ECLAT(data=df_copy.iloc[:1000], verbose=True)
rule_indices, frequent_itemsets = eclat_instance.fit(min_support=0.5, min_combination=1, max_combination=3)

frequent_itemsets

100%|████████████████████████████████████████████████████████████████████████████| 5914/5914 [00:02<00:00, 2788.92it/s]


ValueError: Cannot index with multidimensional key